In [190]:
import datetime
import pyodbc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

## Всем привет! Здесь описано подключение к БД, примеры запросов и тд.

### Обратите внимание на признак Probability - его использование зависит от выбранного вами направления. 
### Просим в явном виде отображать его исключение в 1 направлении, мы будем каждый ноутбук проверять и прогонять модели руками (просьба не мухлевать и не использовать информацию из будущего в признаках - мы все заметим :) )

## Формат выгрузки csv-файла (столбцы):
### Для чек-поинтов - это на тренировочных данных, для финального резульата - на тестовых данных

1.   LocationId
2.   ProductId
3.   ValidationDateTime
4.   CalculatedProbability (вычисленная вами вероятность - predict_proba() )

### Также будет очень приятно, если вы нам выведите feature_importances в виде картиночки :)

In [191]:
conn_str = """
    Driver={ODBC Driver 17 for SQL Server};
    Server=109.71.11.235;
    Database=GFC.Hackathon;
    UID=participant;
    PWD=AiNRYmEatiBL;
"""

conn = pyodbc.connect(conn_str)

In [214]:
data_query = """
SELECT * FROM dbo.Features 
"""

In [139]:
query = """
SELECT
LocationId,
ProductId,
ValidationDateTime,
IsCorrect,
LAG(IsCorrect, 1) OVER (PARTITION BY LocationId, ProductId ORDER BY ValidationDateTime) AS lag_1_iscorrect,
LAG(IsCorrect, 2) OVER (PARTITION BY LocationId, ProductId ORDER BY ValidationDateTime) AS lag_2_iscorrect,
LAG(IsCorrect, 3) OVER (PARTITION BY LocationId, ProductId ORDER BY ValidationDateTime) AS lag_3_iscorrect,
LAG(IsCorrect, 4) OVER (PARTITION BY LocationId, ProductId ORDER BY ValidationDateTime) AS lag_4_iscorrect,
LAG(IsCorrect, 5) OVER (PARTITION BY LocationId, ProductId ORDER BY ValidationDateTime) AS lag_5_iscorrect
FROM dbo.Features s
ORDER BY LocationId, ProductId, ValidationDateTime;
"""

In [ ]:
last_price_query = """
        SELECT
        s.IsCorrect ,
        s.LocationId ,
        s.ProductId ,
        s.ValidationDateTime ,
        p.Datetime ,
        p.PriceSum,
        p.Quantity
        FROM dbo.Features AS s
        LEFT JOIN dbo.LocationStateHourSales AS p
        ON s.LocationId  = p.LocationId
        AND s.ProductId  = p.ProductId
        AND p.Datetime  < s.ValidationDateTime
        ORDER BY ValidationDateTime DESC, Datetime DESC
        """

In [81]:
query_features_HourSales = """
with history_transactions as (
        SELECT
        s.IsCorrect ,
        s.LocationId ,
        s.ProductId ,
        s.ValidationDateTime ,
        p.Datetime ,
        p.PriceSum,
        p.Quantity
        FROM dbo.Features AS s
        LEFT JOIN dbo.LocationStateHourSales AS p
        ON s.LocationId  = p.LocationId
        AND s.ProductId  = p.ProductId
        AND p.Datetime  < s.ValidationDateTime)
        --ORDER BY ValidationDateTime DESC, Datetime DESC)
    SELECT  IsCorrect, LocationId,
ProductId,
ValidationDateTime,
--Цена
AVG(PriceSum) as mean_price,
max(PriceSum) as max_price,
min(PriceSum) as min_price,
max(PriceSum) - min(PriceSum) as range_price,
--VARIANCE(PriceSum) as variance_price,
SUM(PriceSum) as sum_price,
--Количество
AVG(Quantity) as mean_quantity,
max(Quantity) as max_quantity,
min(Quantity) as min_quantity,
max(Quantity) - min(Quantity) as range_quantity,
--VARIANCE(Quantity) as variance_quantity,
SUM(PriceSum) as sum_quantity,
--Количество вхождений
count(*) as cnt
FROM history_transactions
GROUP BY LocationId, ProductId, ValidationDateTime, IsCorrect
"""

In [202]:
query_features_Stocks = """
with history_transactions as (
        SELECT
        s.LocationId ,
        s.ProductId ,
        s.ValidationDateTime ,
        p.Datetime ,
        p.Quantity
        FROM dbo.Features AS s
        LEFT JOIN dbo.LocationStateStocks AS p
        ON s.LocationId  = p.LocationId
        AND s.ProductId  = p.ProductId
        AND p.Datetime  < s.ValidationDateTime)
        --ORDER BY ValidationDateTime DESC, Datetime DESC)
    SELECT  LocationId,
ProductId,
ValidationDateTime,
--Количество
AVG(Quantity) as mean_quantity,
max(Quantity) as max_quantity,
min(Quantity) as min_quantity,
max(Quantity) - min(Quantity) as range_quantity,
--VARIANCE(Quantity) as variance_quantity,
SUM(Quantity) as sum_quantity,
--Количество вхождений
count(*) as cnt
FROM history_transactions
GROUP BY LocationId, ProductId, ValidationDateTime
"""

In [203]:
# пример запросов - здесь выведен весь датасет признаков
df = pd.read_sql(query_features_Stocks, conn)
df.to_csv('stocks_features.csv')

In [ ]:
df = pd.read_sql(last_price_query, conn)
df_not_duplicates = df.drop_duplicates(subset=(['LocationId', 'ProductId', 'ValidationDateTime']))
df_not_duplicates.to_csv('last_price_quant.csv')


In [215]:
import pandas as pd
df = pd.read_sql(data_query, conn) #Стандартный файл с выгрузкой с dbeaver, который был в телеграмме изначально использовался, но в последний момент заменяю на запрос. Возможно будут дубликаты
df = df.sort_values(by=['LocationId', 'ProductId', 'ValidationDateTime'])

In [217]:
df['lag_1_5_mean'] = df.iloc[:,4:9].mean(axis=1)
df['lag_1_2_mean'] = df.iloc[:,4:6].mean(axis=1)
df['lag_1_3_mean'] = df.iloc[:,4:7].mean(axis=1)
df['lag_1_4_mean'] = df.iloc[:,4:8].mean(axis=1)
df['lag_2_5_mean'] = df.iloc[:,5:9].mean(axis=1)
df['lag_3_5_mean'] = df.iloc[:,6:9].mean(axis=1)
df['lag_4_5_mean'] = df.iloc[:,7:9].mean(axis=1)

In [71]:
df['lag_1_5_var'] = df.iloc[:, 4:9].var(axis=1)
df['lag_1_2_var'] = df.iloc[:, 4:6].var(axis=1)
df['lag_1_3_var'] = df.iloc[:, 4:7].var(axis=1)
df['lag_1_4_var'] = df.iloc[:, 4:8].var(axis=1)
df['lag_2_5_var'] = df.iloc[:, 5:9].var(axis=1)
df['lag_3_5_var'] = df.iloc[:, 6:9].var(axis=1)
df['lag_4_5_var'] = df.iloc[:, 7:9].var(axis=1)

In [80]:
df['lag_1_5_std'] = df.iloc[:, 4:9].std(axis=1)
df['lag_1_2_std'] = df.iloc[:, 4:6].std(axis=1)
df['lag_1_3_std'] = df.iloc[:, 4:7].std(axis=1)
df['lag_1_4_std'] = df.iloc[:, 4:8].std(axis=1)
df['lag_2_5_std'] = df.iloc[:, 5:9].std(axis=1)
df['lag_3_5_std'] = df.iloc[:, 6:9].std(axis=1)
df['lag_4_5_std'] = df.iloc[:, 7:9].std(axis=1)

In [82]:
df.to_csv('lag_1_5_mean_std_var.csv')

In [121]:
# пример запросов - здесь выведен весь датасет признаков
df = pd.read_sql(query_features_HourSales, conn)
df.to_csv('features_22.csv')